In [1]:
# design the full graph for integration of 3 different tools
import os
from dotenv import load_dotenv
from pyprojroot import here

load_dotenv()

True

Set the environment variables

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")

# Initialize the tools

### 1.1 Rag Tool Design for unstructured data

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool

EMBEDDING_MODEL = "text-embedding-3-small"
VECTORDB_DIR = "data/airline_policy_vectordb"
K = 2

@tool
def lookup_policy(query: str)->str:
    """Consult the company policies to check whether certain options are permitted."""
    vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=OpenAIEmbeddings(model=EMBEDDING_MODEL)
    )
    docs = vectordb.similarity_search(query, k=K)
    return "\n\n".join([doc.page_content for doc in docs])

print(lookup_policy)

name='lookup_policy' description='Consult the company policies to check whether certain options are permitted.' args_schema=<class 'langchain_core.utils.pydantic.lookup_policy'> func=<function lookup_policy at 0x763f1410b2e0>


In [4]:
VECTORDB_DIR = "data/airline_policy_vectordb"
K=2

vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=EMBEDDING_MODEL
)
print("Number of vectors in vectordb:",
      vectordb._collection.count(), "\n\n")


Number of vectors in vectordb: 22 




In [5]:
lookup_policy.invoke("can I cancel my ticket?")

"conditions of your ticket to make sure you're eligible for a refund.\nSwiss Air Cancellation Fees The cancellation fees for Swiss Air flights may vary depending on the type of ticket you have\npurchased. The airline offers three different types of tickets, which are:\nFlex Ticket - This ticket offers the most flexibility and allows passengers to make changes or cancel their flight without\nincurring any additional fees. Standard Ticket - This ticket offers some flexibility but charges a fee for changes and\ncancellations. The fee for cancelling a standard ticket can range from 100 to 250 CHF, depending on the route and time\nof cancellation. Economy Ticket - This ticket offers the least flexibility and charges a fee for any changes or cancellations.\nThe fee for cancelling an economy ticket can range from 150 to 350 CHF, depending on the route and time of\ncancellation. It is important to note that the fees listed above are subject to change and may vary depending on the\nspecific tic

### 1.2 Search tool design

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
search_tool = TavilySearchResults(max_results=2)
search_tool.invoke("what is latest news of todays from india")

[{'url': 'https://timesofindia.indiatimes.com/india/timestopten.cms',
  'content': 'With legal sand extraction unable to meet demand, illegal mining has surged, targeting riverbeds and resulting in environmental degradation and threats to human life, reports ToI. In states like Madhya Pradesh and Haryana, villagers, police officers, and government officials who oppose illegal sand mining face violent reprisals. In 2022, Haryana witnessed 12 sand mining-related attacks, including the murder of DSP Surender Singh, who was run over by a dumper truck during an illegal mining investigation, with six suspects arrested and the case still ongoing. Follow news that matters to you in real-time. A fuel tanker collided head-on with another truck in Nigeria on Sunday causing an explosion that killed at least 48 people, the country’s emergency response agency said.'},
 {'url': 'https://www.ndtv.com/india',
  'content': 'Before Poll Panel, Sharad Pawar Camp Questions Premise Of Dispute In NCP\nThe Sh

In [9]:
### 1.3 Sql Agent tool design 
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# SQL AGent chain

In [ ]:
sqldb_directory = here("../data/Chinook.db")
